In [4]:
import sys
#import lab_equipment as instruments,  signal_recovery_7270 as SR7270 #Importing Librarys of the instruments will be used
from time import sleep
import time
import pyvisa
import os
import numpy as np
#import pymeasure
#from pymeasure.instruments.oxfordinstruments import ITC503

In [2]:
def find_TC(f):
    TC=np.array([1e-5,2e-5,5e-5,1e-4,2e-4,5e-4,1e-3,2e-3,5e-3,1e-2,2e-2,5e-2,1e-1,2e-1,5e-1,1,2,5,10,20,50,100,200,500,1e3,2e3,5e3,1e4,2e4,5e4,1e5])
    aux=abs(TC-10/f).tolist()
    ind = aux.index(min(aux))
    return ind

def measurement_time(f):
    if f<1:
        time = int(100/f)
        return time
    else: return 100
    
def change_sen(id, V):
    sen= float(id.query('SEN.')[0:-2])
    aux= int(id.query('SEN')[0:-2])
    if V>sen:
        SR7270.set_sen(id,aux+1)
        sleep(1)
    if V<0.25*sen:
        SR7270.set_sen(id,aux-1)
        sleep(1)

def create_file(file_name, directory):
    file = open(directory + file_name + '.dat', "w")                                         #Abre arquivo de data
    file.write('[Header]\nTIMEMODE, SECONDS, RELATIVE\nTITLE, Python Log Data File \n')      #Header for MultiVu
    file.write('[Data]\n Time (s), Temperature (K), Voltage Lockin 1 (V), X Lockin 1 (V), Y Lockin 1,\
                Phase 1, Voltage Lockin 2 (V), X Lockin 2 (V), Y Lockin 2, Phase 2, Sensor Resistance (Ohm) \n')     

def write2file(file_name, directory, time, T, V1, Vx1, Vy1, ph1, V2, Vx2, Vy2, ph2, R):
    file = open(directory + file_name + '.dat', "a")
    for i in range(0, 1):
        file.write(str(time) + "," +
                   str(T) + "," +
                   str(V1) + ","+
                   str(Vx1) + ","+
                   str(Vy1) + ","+
                   str(ph1) + ","+
                   str(V2) + ","+
                   str(Vx2) + ","+
                   str(Vy2) + ","+
                   str(ph2) + ","+
                   str(R) + "\n")
    os.fsync(file)
    file.flush()

def create_file2(file_name, directory):
    file = open(directory + file_name + '.dat', "w")                                         #Abre arquivo de data
    file.write('[Header]\nTIMEMODE, SECONDS, RELATIVE\nTITLE, Python Log Data File \n')      #Header for MultiVu
    file.write('[Data]\n Temperature (K), Voltage Lockin 1 (V), X Lockin 1 (V), Y Lockin 1, Phase 1,\
               Voltage Lockin 2 (V), X Lockin 2 (V), Y Lockin 2, Phase 2, Sensor Resistance (Ohm) \n')
    
def write2file2(file_name, directory, T, V1, Vx1, Vy1, ph1, V2, Vx2, Vy2, ph2, R):
    file = open(directory + file_name + '.dat', "a")
    for i in range(0, 1):
        file.write(str(T) + "," +
                   str(V1) + "," +
                   str(Vx1) + "," +
                   str(Vy1) + ","+
                   str(ph1) + ","+
                   str(V2) + ","+
                   str(Vx2) + ","+
                   str(Vy2) + ","+
                   str(ph2) + ","+
                   str(R) + "\n")
    os.fsync(file)
    file.flush()

In [7]:
rm = pyvisa.ResourceManager()
rm.list_resources('?*')

('USB0::0x0A2D::0x001B::09295879::RAW',
 'USB0::0x0AAD::0x01E6::101296::INSTR',
 'ASRL1::INSTR',
 'ASRL3::INSTR',
 'ASRL4::INSTR',
 'ASRL5::INSTR')

In [6]:
SR1= rm.open_resource('USB0::0x0A2D::0x001B::12208162::RAW')
SR2= rm.open_resource('USB0::0x0A2D::0x001B::12202815::RAW')
LS= rm.open_resource('GPIB0::11::INSTR')
VTI = rm.open_resource('GPIB0::22::INSTR')
VTI.read_termination='\r'

VisaIOError: VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.

In [ ]:
#adapter = pymeasure.adapters.VISAAdapter('GPIB0::22::INSTR')
#VTI = ITC503(adapter)

In [30]:

# VTI.write('T4.50')

# stp = VTI.query('R0\r')[1:-1]
# stp

In [5]:
def check_number(n):
    try:
        float(n)
        if float(n)>500 or float(n)==679 or float(n)==2.8:
            return False
        else: 
            return float(n)
    except ValueError:
        return False

In [10]:
directory='C:/Users/kelvinox/Fernando/p3/0T/temperature_sweeps/'
filename='2024-04-04_thermocouple_rampdown_9p1Hz_3mA'

In [11]:
### Start Temperature ramp
### LOCKIN AMPLIFIERS
create_file(filename, directory)
time_i=time.time()
ST=25
s=1
i=0
while ST>1.2:
    sleep(1)
    V1=SR7270.query_mag(SR1)
    Vx1=SR7270.query_x(SR1)
    Vy1=SR7270.query_y(SR1)
    ph1=SR7270.query_phase(SR1)

    V2=SR7270.query_mag(SR2)
    Vx2=SR7270.query_x(SR2)
    Vy2=SR7270.query_y(SR2)
    ph2=SR7270.query_phase(SR2)
            
    t=time.time()-time_i
    T = float(LS.query('KRDG?')[0:-2])
    #R = float(LS.query('RDGR? 1')[0:-2])
    n = VTI.query('R1')[1:]
    m = check_number(n)
    if m==False:
        TVTI = n
    else:
        TVTI = m
    
    if i%10==0:
        change_sen(SR1, V1)
        change_sen(SR2, V2)
    i=i+1
    write2file(filename, directory, t, T, V1, Vx1, Vy1, ph1, V2, Vx2, Vy2, ph2, TVTI)
    s=s+1
    if i%15==0:
        ST=ST-0.05
        VTI.write(f'T{ST}')

In [12]:
sleep(1800)

In [13]:
#directory='C:/Users/kelvinox/Fernando/temperature_sweeps/'
filename='2024-04-05_thermocouple_ramp2_9p1Hz_3mA'

In [14]:
### Start Temperature ramp
### LOCKIN AMPLIFIERS
create_file(filename, directory)
time_i=time.time()
ST=1.2
s=1
i=0
while ST<25:
    sleep(1)
    V1=SR7270.query_mag(SR1)
    Vx1=SR7270.query_x(SR1)
    Vy1=SR7270.query_y(SR1)
    ph1=SR7270.query_phase(SR1)

    V2=SR7270.query_mag(SR2)
    Vx2=SR7270.query_x(SR2)
    Vy2=SR7270.query_y(SR2)
    ph2=SR7270.query_phase(SR2)
            
    t=time.time()-time_i
    T = float(LS.query('KRDG?')[0:-2])
    #R = float(LS.query('RDGR? 1')[0:-2])
    n = VTI.query('R1')[1:]
    m = check_number(n)
    if m==False:
        TVTI = n
    else:
        TVTI = m
    
    if i%10==0:
        change_sen(SR1, V1)
        change_sen(SR2, V2)
    i=i+1
    write2file(filename, directory, t, T, V1, Vx1, Vy1, ph1, V2, Vx2, Vy2, ph2, TVTI)
    s=s+1
    if i%15==0:
        ST=ST+0.1
        VTI.write(f'T{ST}')

In [27]:
### Start the T sweep
### LOCKIN AMPLIFIERS
create_file2(filename, directory)
time_i=time.time()
ST=1.2
s=1
while ST<25:
    VTI.write(f'T{ST}')
    sleep(1)
    create_file(f'T{int(s)}', directory+filename+'/')
    Vmean1, Vmean2 = [], []
    Xmean1, Xmean2 = [], []
    Ymean1, Ymean2 = [], []
    Pmean1, Pmean2 = [], []
    Tmean = []
    i=0
    while i<300:
        V1=SR7270.query_mag(SR1)
        Vx1=SR7270.query_x(SR1)
        Vy1=SR7270.query_y(SR1)
        ph1=SR7270.query_phase(SR1)
        Vmean1.append(V1), Xmean1.append(Vx1), Ymean1.append(Vy1), Pmean1.append(ph1)

        V2=SR7270.query_mag(SR2)
        Vx2=SR7270.query_x(SR2)
        Vy2=SR7270.query_y(SR2)
        ph2=SR7270.query_phase(SR2)
        Vmean2.append(V2), Xmean2.append(Vx2), Ymean2.append(Vy2), Pmean2.append(ph2)
            
        t=time.time()-time_i
        T = float(LS.query('KRDG?')[0:-2])
        Tmean.append(T)
        #R = float(LS.query('RDGR? 1')[0:-2])
        n = 1#VTI.query('R1')[1:]
        m = check_number(n)
        if m==False:
            TVTI=TVTI
        else:
            TVTI = m
        write2file(f'T{int(s)}', directory+filename+'/', t, T, V1, Vx1, Vy1, ph1, V2, Vx2, Vy2, ph2, TVTI)
        if i%15==0:
            change_sen(SR1, V1)
            change_sen(SR2, V2)
        sleep(1)
        i=i+1
    V1, Vx1, Vy1, ph1 = np.mean(Vmean1[-int(len(Vmean1)*0.10):-1]), np.mean(Xmean1[-int(len(Xmean1)*0.10):-1]), np.mean(Ymean1[-int(len(Ymean1)*0.10):-1]), np.mean(Pmean1[-int(len(Pmean1)*0.10):-1]) 
    V2, Vx2, Vy2, ph2 = np.mean(Vmean2[-int(len(Vmean2)*0.10):-1]), np.mean(Xmean2[-int(len(Xmean2)*0.10):-1]), np.mean(Ymean2[-int(len(Ymean2)*0.10):-1]), np.mean(Pmean2[-int(len(Pmean2)*0.10):-1])
    T = np.mean(Tmean[-int(len(Tmean)*0.10):-1])
    write2file2(filename, directory, T, V1, Vx1, Vy1, ph1, V2, Vx2, Vy2, ph2, TVTI)
    s=s+1
    ST=ST+0.01

In [ ]:
### Start the T sweep
### RESISTANCE BRIDGE
create_file2(filename, directory)
time_i=time.time()
ST=1.56
s=1
while ST<6.95:
    VTI.write(f'T{ST}')
    sleep(1)
    create_file(f'T{int(s)}', directory+'T-scan10/')
    Vmean1, Vmean2 = [], []
    Xmean1, Xmean2 = [], []
    Ymean1, Ymean2 = [], []
    Pmean1, Pmean2 = [], []
    Tmean = []
    i=0
    while i<30:
        LS.write('SCAN 2,1')
        sleep(10)
        V1=float(LS.query('RDGR? 2')[0:-2])#SR7270.query_mag(SR1)
        Vx1=1#SR7270.query_x(SR1)
        Vy1=1#SR7270.query_y(SR1)
        ph1=1#SR7270.query_phase(SR1)
        Vmean1.append(V1), Xmean1.append(Vx1), Ymean1.append(Vy1), Pmean1.append(ph1)

        V2=1#SR7270.query_mag(SR2)
        Vx2=1#SR7270.query_x(SR2)
        Vy2=1#SR7270.query_y(SR2)
        ph2=1#SR7270.query_phase(SR2)
        Vmean2.append(V2), Xmean2.append(Vx2), Ymean2.append(Vy2), Pmean2.append(ph2)
            
        t=time.time()-time_i
        LS.write('SCAN 1,1')
        sleep(10)
        T = float(LS.query('RDGK? 1')[0:-2])
        Tmean.append(T)
        R = float(LS.query('RDGR? 1')[0:-2])
        write2file(f'T{int(s)}', directory+'T-scan10/', t, T, V1, Vx1, Vy1, ph1, V2, Vx2, Vy2, ph2, R)
        #if i%15==0:
            #change_sen(SR1, V1)
            #change_sen(SR2, V2)
        #sleep(1)
        i=i+1
    V1, Vx1, Vy1, ph1 = np.mean(Vmean1[-int(len(Vmean1)*0.10):-1]), np.mean(Xmean1[-int(len(Xmean1)*0.10):-1]), np.mean(Ymean1[-int(len(Ymean1)*0.10):-1]), np.mean(Pmean1[-int(len(Pmean1)*0.10):-1]) 
    V2, Vx2, Vy2, ph2 = np.mean(Vmean2[-int(len(Vmean2)*0.10):-1]), np.mean(Xmean2[-int(len(Xmean2)*0.10):-1]), np.mean(Ymean2[-int(len(Ymean2)*0.10):-1]), np.mean(Pmean2[-int(len(Pmean2)*0.10):-1])
    T = np.mean(Tmean[-int(len(Tmean)*0.10):-1])
    write2file2(filename, directory, T, V1, Vx1, Vy1, ph1, V2, Vx2, Vy2, ph2, R)
    s=s+1
    ST=ST+0.01

In [15]:
directory='C:/Users/kelvinox/Fernando/p3/0T/temperature_sweeps/'
filename='2024-04-11_warmingup'

In [23]:
#monitoring
T = float(LS.query('KRDG?')[0:-2])
n = VTI.query('R1')[1:]
m = check_number(n)
if m==False:
    TVTI=0
else:
    TVTI = m
#create_file(filename, directory)
#time_i=time.time()
i=0
while T<300:
    V1=SR7270.query_mag(SR1)
    Vx1=SR7270.query_x(SR1)
    Vy1=SR7270.query_y(SR1)
    ph1=SR7270.query_phase(SR1)

    V2=1#SR7270.query_mag(SR2)
    Vx2=1#SR7270.query_x(SR2)
    Vy2=1#SR7270.query_y(SR2)
    ph2=1#SR7270.query_phase(SR2)

    t=time.time()-time_i
    T = float(LS.query('KRDG?')[0:-2])
    R = 1#float(LS.query('RDGR?')[0:-2])
    n = VTI.query('R1')[1:]
    m = check_number(n)
    if m==False:
        TVTI=TVTI
    else:
        TVTI = m
    if i%15==0:
        change_sen(SR1, V1)
        #change_sen(SR2, V2)
    i+=1
    write2file(filename, directory, t, T, V1, Vx1, Vy1, ph1, V2, Vx2, Vy2, ph2, TVTI)
    sleep(1)

KeyboardInterrupt: 

In [ ]:
### Noise test
directory='C:/Users/kelvinox/Fernando/'
file_name='Pb_noise-test_lowT2(Ground,FET)'
create_file(file_name, directory)
time_i=time.time()
Vmean1, Vmean2 = [], []
Xmean1, Xmean2 = [], []
Ymean1, Ymean2 = [], []
Pmean1, Pmean2 = [], []
for i in range (300):
        V1=SR7270.query_mag(SR1)
        Vx1=SR7270.query_x(SR1)
        Vy1=SR7270.query_y(SR1)
        ph1=float(SR1.query('PHA.')[0:-2])
        Vmean1.append(V1), Xmean1.append(Vx1), Ymean1.append(Vy1), Pmean1.append(ph1)

        V2=SR7270.query_mag(SR2)
        Vx2=SR7270.query_x(SR2)
        Vy2=SR7270.query_y(SR2)
        ph2=float(SR2.query('PHA.')[0:-2])
        Vmean2.append(V2), Xmean2.append(Vx2), Ymean2.append(Vy2), Pmean2.append(ph2)

        t=time.time()-time_i
        T = float(LS.query('RDGK?')[0:-2])
        R = float(LS.query('RDGR?')[0:-2])
        write2file(file_name, directory, t, T, V1, Vx1, Vy1, ph1, V2, Vx2, Vy2, ph2, R)
        sleep(1)

In [ ]:
print(np.mean(Vmean1), np.std(Vmean1))
print(np.mean(Vmean2), np.std(Vmean2))

In [24]:
SR7270.set_v(SR1,1.5)
SR7270.set_v(SR2,1.5)

In [ ]:
rm.close()